In [273]:
from __future__ import division, print_function # Imports from __future__ since we're running Python 2
import os
import numpy as np
import collections as cls
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import itertools
import math
import glob
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from matplotlib import pylab
from pylab import *
random_state = 10

In [274]:
#get all city names
all_cities_paths =  glob.glob(os.path.join(os.getcwd(), 'data','*'))
cities = []
for city in all_cities_paths:
    cities.append(city.split("/")[-1])
print(cities)
                    

['amsterdam', 'berlin', 'copenhagen', 'dublin', 'edinburgh', 'helsinki', 'lisbon', 'london', 'madrid', 'moscow', 'reykiavik', 'rome', 'tallin', 'vienna', 'paris']


In [275]:
#loading each city's dataset to a dictionary
datasets = dict()
for city in cities:
    data_path = glob.glob(os.path.join(os.getcwd(), 'data',city,city,city,'*.csv')) 
    for f in data_path:
        city_data =pd.read_csv(f,delimiter=",",lineterminator="\n") 
        city_data.fillna("(unknown)",inplace=True)
        datasets[city] = city_data
        

In [276]:
#find out which of the columns of each city's dataset have missing values
city_columns = dict() #it is going to conatin the feature list of each city
for city,data in datasets.iteritems():
    columns = data.columns.values
    city_columns[city] = list(columns)
    some_mis = []
    only_mis = []
    for col in columns:
        bools = data[col].to_frame().isin({col: ['(unknown)']}) 
        num = len(bools[bools[col] == True].index) #number of mssing values in the particular column
        if num > 0: #if there is at least one missing value
            if num < len(bools.index): #if not all values are missing
                some_mis.append(col)
            else:
                only_mis.append(col) #if all values of the column are missing
    print_str = '\n{} '.format(city)
    if len(some_mis) > 0:
        print_str += '\ncols with some missing values--> {}'.format(some_mis[0])
        for i in range(1,len(some_mis)):
            print_str += ',{}'.format(some_mis[i])
    if len(only_mis) > 0:
        print_str += '\ncols with only missing values--> {}'.format(only_mis[0])
        for i in range(1,len(only_mis)):
            print_str += ',{}'.format(only_mis[i])
    print(print_str)
        
    


edinburgh 
cols with some missing values--> name
cols with only missing values--> country,borough,bathrooms,minstay

helsinki 
cols with some missing values--> name
cols with only missing values--> country,borough,bathrooms,minstay

paris 
cols with some missing values--> overall_satisfaction
cols with only missing values--> country,borough,bathrooms,minstay

madrid 
cols with only missing values--> minstay

moscow 
cols with only missing values--> country,bathrooms,minstay

dublin 
cols with only missing values--> country,borough,bathrooms,minstay

tallin 
cols with only missing values--> country,borough,bathrooms,minstay

berlin 
cols with some missing values--> name
cols with only missing values--> country,borough,bathrooms,minstay

rome 
cols with only missing values--> country,borough,bathrooms,minstay

london 
cols with some missing values--> overall_satisfaction
cols with only missing values--> country,bathrooms,minstay

lisbon 
cols with some missing values--> name
cols with o

In [277]:
datasets['madrid'].sample(5)

,room_id,host_id,room_type,borough,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,price,minstay,latitude,longitude,last_modified
7650,13997848,44682175,Entire home/apt,Centro,Universidad,15,4.5,2,0.0,38.0,(unknown),40.424630,-3.703220,2017-03-12 01:21:26.770421\r
9512,17615698,25342839,Private room,Carabanchel,Buenavista,0,0.0,1,1.0,11.0,(unknown),40.367509,-3.756273,2017-03-11 17:39:24.680104\r
10304,4656142,24099435,Private room,Arganzuela,Delicias,85,4.5,3,1.0,33.0,(unknown),40.397675,-3.693506,2017-03-11 15:46:31.287236\r
6222,1532354,7218465,Entire home/apt,Centro,Embajadores,38,4.5,3,1.0,49.0,(unknown),40.408426,-3.706267,2017-03-12 01:45:45.601239\r
2402,2019799,147247,Entire home/apt,Retiro,Jerónimos,3,4.5,2,1.0,99.0,(unknown),40.409417,-3.690223,2017-03-12 09:31:18.713692\r


In [278]:
#find out which columns are existent to all of the cities and which are not
all_columns = set() # the union of columns appearing in all of the cities
all_common_columns = set() # the intersection of columns appearing in all of the cities
first = True
for city,columns in city_columns.iteritems():
    columns_set = set(columns) #since all the columns of the same city have a unique name
    for col in columns:
        all_columns.add(col.strip())
    if first == False:
        mortals = [] #containing the columns that are going to be removed from the intersection
        for col in all_common_columns:
            if col not in columns_set:
                mortals.append(col)
        for mor in mortals: 
            all_common_columns.remove(mor)
    else:
        for col in columns:
            all_common_columns.add(col.strip()) #initializing the intersection
        first = False

all_common_list = list(all_common_columns) #set to list
if len(all_common_list) > 0:
    print_str = 'Features appearing to every city of the dataset:\n{}'.format(all_common_list[0])
    for i in range(1,len(all_common_list)): #the rest
        print_str += ',{}'.format(all_common_list[i])
    print(print_str)
else:
    print("There are no common features between the different cities")


print_str = "\nFeatures appearing only to some cities of the dataset:\n"
first = True
for col in all_columns:
    if(col not in all_common_columns):
        if first == True:
            print_str += col
            first = False
        else:
            print_str += "," + col

print(print_str)


    

Features appearing to every city of the dataset:
borough,neighborhood,bedrooms,overall_satisfaction,longitude,reviews,latitude,room_id,accommodates,minstay,price,host_id,room_type

Features appearing only to some cities of the dataset:
property_type,city,name,country,bathrooms,last_modified,survey_id,location


In [279]:
#checking if some bad values exist in accommodates column for every city
bad_accommodates_value_found = False
for city,data in datasets.iteritems():
    if len(data[data['accommodates'] < 1].index) > 0:
        bad_accommodates_value_found = True
    
if bad_accommodates_value_found:
    print("There are bad values in accommodates' column")
else:
    print("There are not bad values in accommodates' column ")

There are not bad values in accommodates' column 


In [280]:
#create new column: price per number of people that can be accommodated

for city in datasets.keys():
    print(city)
    city_data = datasets[city]
    for index , row in city_data.iterrows():
        city_data.set_value(index,'price_per_accommodated',row['price']/row['accommodates'])
    datasets[city] = city_data

edinburgh
helsinki
paris
madrid
moscow
dublin
tallin
berlin
rome
london
lisbon
amsterdam
copenhagen
vienna


In [281]:
#input: p--> price of property, quartiles --> quartiles of price dataset,
#k --> tukey test's hyperparameter, status --> sale or rental
#output: whether p is an outlier is an low outlier, a high outlier or normal instance
def tukey_test(p,quartiles,k):
    iqr = quartiles[2] - quartiles[0]
    lower_bound = quartiles[0] - k * iqr
    if(lower_bound < 1):
        lower_bound = 1
        
    upper_bound  = quartiles[2] + k * iqr
    low_outliers = 0
    high_outliers = 0
    if p < lower_bound:
        return "low_outlier"
    elif p > upper_bound:
        return "high_outlier"
    return "no" 

In [282]:
#given the fact that some prices might be per month outlier detection method tukey test will be used
#so that these entries will be captures as outliers

k = 1.5 #tukey test's hyperparameter
all_low_outliers = 0
all_high_outliers = 0
all_data = 0
for city in datasets.keys():
    city_data = datasets[city]
    quartiles = []
    quartiles.append(city_data['price_per_accommodated'].to_frame().quantile(q=0.25,axis=0)[0])
    quartiles.append(city_data['price_per_accommodated'].to_frame().mean(axis=0)[0])
    quartiles.append(city_data['price_per_accommodated'].to_frame().quantile(q=0.75,axis=0)[0])
    mortals = [] #create appends to mortals
    city_low_outliers = 0
    city_high_outliers = 0
    for index , row in city_data.iterrows():
        res = tukey_test(row['price_per_accommodated'],quartiles,k)
        if res == 'low_outlier':
            city_low_outliers += 1
        elif res == 'high_outlier':
            city_high_outliers += 1
    all_data += len(city_data.index)
    all_low_outliers += city_low_outliers
    all_high_outliers += city_high_outliers
    print('{0}\t(LOW)\t{1}\t(HIGH)\t{2}\t(TOTAL)\t{3}\t{4}% of city\'s dataset'.
          format(city,city_low_outliers,city_high_outliers,city_low_outliers + city_high_outliers
                 ,100 * float(city_low_outliers + city_high_outliers)/len(city_data.index)))

print('\n\n\nALTOGETHER\t(LOW)\t{0}\t(HIGH)\t{1}\t(TOTAL)\t{2}\t{3}% of all cities\' datasets'.
          format(all_low_outliers,all_high_outliers,all_low_outliers + all_high_outliers
                 ,100 * float(all_low_outliers + all_high_outliers)/all_data))
            
            
    
    

edinburgh	(LOW)	0	(HIGH)	318	(TOTAL)	318	4.18586284059% of city's dataset
helsinki	(LOW)	0	(HIGH)	157	(TOTAL)	157	5.34741144414% of city's dataset
paris	(LOW)	0	(HIGH)	3173	(TOTAL)	3173	4.98632806361% of city's dataset
madrid	(LOW)	0	(HIGH)	660	(TOTAL)	660	5.23186682521% of city's dataset
moscow	(LOW)	80	(HIGH)	441	(TOTAL)	521	7.67417881868% of city's dataset
dublin	(LOW)	0	(HIGH)	460	(TOTAL)	460	6.36413945766% of city's dataset
tallin	(LOW)	1	(HIGH)	72	(TOTAL)	73	4.27400468384% of city's dataset
berlin	(LOW)	0	(HIGH)	888	(TOTAL)	888	4.51196585539% of city's dataset
rome	(LOW)	0	(HIGH)	1363	(TOTAL)	1363	5.22923460579% of city's dataset
london	(LOW)	0	(HIGH)	2714	(TOTAL)	2714	4.69339051638% of city's dataset
lisbon	(LOW)	3	(HIGH)	779	(TOTAL)	782	6.11080722044% of city's dataset
amsterdam	(LOW)	0	(HIGH)	625	(TOTAL)	625	3.63350967967% of city's dataset
copenhagen	(LOW)	0	(HIGH)	473	(TOTAL)	473	2.54041570439% of city's dataset
vienna	(LOW)	0	(HIGH)	393	(TOTAL)	393	4.9063670412% of city's d

In [260]:
#given the fact that some prices might be per month outlier detection method tukey test will be used
#so that these entries will be captures as outliers
print(datasets['madrid']['price'].to_frame().median(axis=0))
print(datasets['madrid']['price'].to_frame().mean(axis=0))
print(datasets['madrid']['price'].to_frame().quantile(q=0.25,axis=0))
print(datasets['madrid']['price'].to_frame().quantile(q=0.5,axis=0))
print(datasets['madrid']['price'].to_frame().quantile(q=0.75,axis=0))
    

price    60.0
dtype: float64
price    76.70107
dtype: float64
price    38.0
Name: 0.25, dtype: float64
price    60.0
Name: 0.5, dtype: float64
price    93.0
Name: 0.75, dtype: float64


In [240]:
datasets['madrid']

,room_id,host_id,room_type,borough,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,price,minstay,latitude,longitude,last_modified
0,10914818,10940437,Shared room,Centro,Sol,3,3.5,6,1.0,72.0,(unknown),40.414639,-3.707379,2017-03-12 21:39:58.887658\r
1,2143961,10940437,Shared room,Centro,Sol,39,4.5,6,1.0,47.0,(unknown),40.415161,-3.707928,2017-03-12 21:39:29.191731\r
2,1512437,1718458,Shared room,Chamberi,Trafalgar,6,4.0,9,1.0,49.0,(unknown),40.430073,-3.703354,2017-03-12 21:39:28.526794\r
3,16656191,97914995,Shared room,Centro,Palacio,0,0.0,12,1.0,42.0,(unknown),40.415412,-3.711277,2017-03-12 21:39:28.303291\r
4,4599821,23622848,Shared room,Centro,Justicia,10,4.0,4,1.0,43.0,(unknown),40.428224,-3.699263,2017-03-12 21:39:25.698105\r
5,15431978,67622677,Shared room,Hortaleza,Canillas,2,0.0,3,1.0,59.0,(unknown),40.462516,-3.650677,2017-03-12 21:39:25.029886\r
6,11946007,63784494,Shared room,Centro,Palacio,7,5.0,3,1.0,49.0,(unknown),40.413614,-3.713262,2017-03-12 21:39:24.922621\r
7,4265459,3591751,Shared room,Arganzuela,Acacias,31,4.5,2,1.0,36.0,(unknown),40.403934,-3.703544,2017-03-12 21:39:19.700477\r
8,9658216,3908623,Shared room,Puente de Vallecas,San Diego,3,4.5,2,1.0,31.0,(unknown),40.391607,-3.669146,2017-03-12 21:39:18.606933\r
9,15516955,78169966,Shared room,Centro,Cortes,9,4.0,2,1.0,28.0,(unknown),40.411371,-3.696087,2017-03-12 21:39:18.257073\r
